In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#İmport Section
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline


# <font color="Purple"> Content </font>
1. [Information](#1)
1. [Load and Check Data](#2)
1. [Distrubition of scores](#3)
1. [Which map was preferred more?](#4)
    * [Which map was preferred more in which event?](#5)
1. [The effect of map selection on winning the game](#6)
1. [Economy DataFrame](#8)

<a id="1" ></a>
### Information
Counter-Strike: Global Offensive (CS:GO) is a multiplayer first-person shooter developed by Valve and Hidden Path Entertainment. It is the fourth game in the Counter-Strike series. Developed for over two years, Global Offensive was released for Windows, macOS, Xbox 360, and PlayStation 3 in August 2012, and for Linux in 2014. Valve still regularly updates the game, both with smaller balancing patches and larger content additions.<br></br>
The game pits two teams, Terrorists and Counter-Terrorists, against each other in different objective-based game modes. The most common game modes involve the Terrorists planting a bomb while Counter-Terrorists attempt to stop them, or Counter-Terrorists attempting to rescue hostages that the Terrorists have captured. There are eight official game modes, all of which have distinct characteristics specific to that mode. The game also has matchmaking support that allows players to play on dedicated Valve servers.<br></br>Global Offensive received positive reviews from critics on release, who praised the game for its gameplay and faithfulness to the Counter-Strike series, though it was criticized for some early features and the differences between the console and PC versions. Since its release, it has drawn in an estimated 11 million players per month, and remains one of the most played games on Valve's Steam platform. In December 2018, Valve transitioned the game to a free-to-play model, focusing on revenue from cosmetic items.

The game has an active esports scene, continuing the history of international competitive play from previous games in the series. Teams compete in professional leagues and tournaments, and Global Offensive is now one of the largest global esports. (wiki)

<a id="2" ></a>
### Load and Check Data

In [ ]:
economy_df=pd.read_csv("../input/csgo-professional-matches/economy.csv",low_memory=False)
picks_df=pd.read_csv("../input/csgo-professional-matches/picks.csv",low_memory=False)
players_df=pd.read_csv("../input/csgo-professional-matches/players.csv",low_memory=False)
results_df=pd.read_csv("../input/csgo-professional-matches/results.csv",low_memory=False)


**results_df:   map scores and team rankings<br>
picks_df:   order of map picks and vetos in the map selection process.<br>
economy_df:   round start equipment value for all rounds played<br>
players_df:   individual performances of players in each map.**

In [ ]:
economy_df.head()

In [ ]:
picks_df.head()

In [ ]:
players_df.head()

In [ ]:
results_df.head()





<font color="blue" size=3>***Our dataset is very large, so we will examine the matches of the first 30 teams in the hltv ranking.For this, we will use the rank 1 and rank 2 features in result_df.***</font>



In [ ]:
results_df=results_df[(results_df["rank_1"]<=30 ) & ( results_df["rank_2"]<=30)]
results_df.head()

In [ ]:
picks_df=picks_df[picks_df.match_id.isin(results_df.match_id.unique())]
picks_df.head()

In [ ]:
economy_df=economy_df[economy_df.match_id.isin(results_df.match_id.unique())]
economy_df.head()

In [ ]:
players_df=players_df[players_df.match_id.isin(results_df.match_id.unique())]
players_df.head()

<a id="3" ></a>
### Distrubition of scores



<font color="blue" size=3>***We will calculate the percentage with which scores the matches played end.***</font>

In [ ]:
scores=pd.DataFrame()
winner1=[]
winner2=[]
loser1=[]
loser2=[]

winner1=results_df[results_df.result_1>=results_df.result_2].result_1.values
loser1=results_df[results_df.result_1>=results_df.result_2].result_2.values

winner2=results_df[results_df.result_2>=results_df.result_1].result_2.values
loser2=results_df[results_df.result_2>=results_df.result_1].result_1.values

In [ ]:
winner = np.concatenate((winner1,winner2))
loser = np.concatenate((loser1,loser2))
scores= pd.DataFrame(np.vstack((winner,loser)).T,columns=['winner','loser'])


In [ ]:
#burayı çaldım
gb = scores.groupby(by=['winner','loser'])['winner'].count()/scores.shape[0]
overtime_percentage = str(round(gb[gb.index.get_level_values(0)!=16].sum()*100,1))+'%'


In [ ]:

gb = round(gb[gb>10**-3]*100,1)

index_plot = np.array(gb.index.get_level_values(0).astype('str'))+'-'+np.array(
    gb.index.get_level_values(1).astype('str'))

fig = go.Figure()
fig.add_trace(go.Scatter(x=index_plot,y=gb.values, name='results'))
fig.update_layout(xaxis_type='category',title='Scores distribution',xaxis_title='Score',yaxis_title='Percentage of matches (%)')

<a id="4" ></a>
### Which map was preferred more ?

In [ ]:
sns.barplot(x=results_df._map.value_counts().index,y=results_df._map.value_counts().values)
plt.xticks(rotation=40)
plt.xlabel("Maps")
plt.ylabel("number of plays")
print(results_df._map.value_counts())
plt.show()

<a id="5" ></a>
### Which map was preferred more in which event?
***There are lots of events so ı choose the 10 most played events***

In [ ]:
events=list(results_df.event_id.unique())
#There are lots of events so ı choose the 10 most played events
events_10={"event_id":results_df["event_id"].value_counts().index,"sum":results_df["event_id"].value_counts()}
events_10=pd.DataFrame(events_10)
events=events_10.event_id.iloc[:10].values
maps=pd.DataFrame()
for i in events:
    maps=pd.DataFrame({"event_id":i,"maps":results_df[results_df.event_id==i]["_map"].value_counts().index,"sum":results_df[results_df.event_id==i]["_map"].value_counts().values})
    sns.barplot(x="sum",y="maps",hue="event_id",data=maps)
    plt.xticks(rotation=45)
    plt.xlabel("number of plays")
    plt.show()
    


<a id="6"></a>
### The effect of map selection on winning the game

In [ ]:
choses=picks_df[['team_1','team_2','match_id', 'event_id','t1_picked_1','t2_picked_1']]
winners=results_df[['team_1', 'team_2', '_map','map_winner','event_id','match_id']]

In [ ]:
#burayı da çaldım
picks_df.rename(columns={"team_1":"team_1_pick","team_2":"team_2_pick" }, inplace=True)

new_df = pd.merge(results_df, picks_df, left_on="match_id", right_on="match_id")

#filter for best of 3's
new_df = new_df[new_df["best_of"] == "3"]

#add new column for winning team name
new_df["winning_team_name"] = ""
new_df.loc[new_df["map_winner"]==1, "winning_team_name"] = new_df["team_1"]
new_df.loc[new_df["map_winner"]==2, "winning_team_name"] = new_df["team_2"]

#create column for name of team who picked the map
new_df["map_picker"] = ""
new_df.loc[new_df["_map"] == new_df["t1_picked_1"], "map_picker"] = new_df["team_1_pick"]
new_df.loc[new_df["_map"] == new_df["t2_picked_1"], "map_picker"] = new_df["team_2_pick"]
new_df.loc[new_df["_map"] == new_df["left_over"], "map_picker"] = "left_over"

#remove decider games
new_df = new_df[new_df["map_picker"] != "left_over"]

#create column for pick win
new_df["pick_win"] = new_df["winning_team_name"] == new_df["map_picker"]

#create column for rank of winning team
new_df["winning_rank"] = 0
new_df.loc[new_df["map_winner"]==1, "winning_rank"] = new_df["rank_1"]
new_df.loc[new_df["map_winner"]==2, "winning_rank"] = new_df["rank_2"]

#create column for rank of losing team
new_df["losing_rank"] = 0
new_df.loc[new_df["map_winner"]==1, "losing_rank"] = new_df["rank_2"]
new_df.loc[new_df["map_winner"]==2, "losing_rank"] = new_df["rank_1"]

#filter for only teams in the top twenty
new_df = new_df[new_df['winning_rank']< 21]
new_df = new_df[new_df['losing_rank']< 21]

#filter for after a certain date
new_df['date_x'] = pd.to_datetime(new_df['date_x'])
new_df = new_df[new_df['date_x'] > "12-31-2016"]

#groupby team
grouper = new_df.groupby("map_picker")["pick_win"].value_counts(normalize=True)
grouper = grouper[grouper.index.isin([True], level=1)]

#Remove redundant index and sort
grouper = grouper.reset_index(level=1, drop=True)
grouper = grouper.sort_values(ascending = False)
grouper = grouper.to_frame()
grouper = grouper.reset_index()

#Add column based on number of games
filter_series = new_df["map_picker"].value_counts()
filter_series = filter_series.to_frame()
filter_series = filter_series.reset_index()
grouper = pd.merge(grouper, filter_series, left_on="map_picker", right_on="index")
grouper = grouper.drop("index", 1)
grouper.rename(columns={"map_picker_x":"Team","pick_win": "Win Percentage", "map_picker_y":"Number of Games" }, inplace=True)
grouper = grouper[grouper["Number of Games"] > 20]
print(grouper)

<a id="8"></a>
## Economy DataFrame,
burayı da çaldım


In [ ]:
#0 Losses: $1400;
#1 Loss: $1900;
#2 Losses: $2400;
#3 Losses: $2900;
#4+ Losses: $3400.
#$100: Sniper rifle kills;
#$300: pistol, assault rifle and grenade kills;
#$600: SMG kills;
#$900: Shotgun kills;
#$1500: Knife kills.
#$ 0-5 K: Eco;
#$ 5-10 K: Forced Pistols;
#$ 10-20 K: Semi-buy;
#$ 20+ K: Full buy.
#$10-15 K: ForcedSMGs;
#$15-20 K: ForcedBuy.

In [ ]:
economy_df.head()

In [ ]:
money_columns = ['2_t1','3_t1','4_t1','5_t1','6_t1','7_t1','8_t1','9_t1','10_t1','11_t1','12_t1','13_t1','14_t1'
                ,'15_t1','17_t1','18_t1','19_t1','20_t1','21_t1','22_t1','23_t1','24_t1','25_t1','26_t1','27_t1',
                 '28_t1','29_t1','30_t1',
                '2_t2','3_t2','4_t2','5_t2','6_t2','7_t2','8_t2','9_t2','10_t2','11_t2','12_t2','13_t2','14_t2'
                ,'15_t2','17_t2','18_t2','19_t2','20_t2','21_t2','22_t2','23_t2','24_t2','25_t2','26_t2','27_t2',
                 '28_t2','29_t2','30_t2']

economy_categories = {0:{'name':'eco','start':0,'end':5000},
                      1:{'name':'forcedPistols','start':5000,'end':10000},
                      2:{'name':'forcedSMGs','start':10000,'end':15000},
                      3:{'name':'forcedBuy','start':15000,'end':20000},
                      4:{'name':'fullBuy','start':20000,'end':50000}
                      }

In [ ]:
for col in money_columns:
    for key, category in economy_categories.items():
        economy_df.loc[(economy_df[col]>category['start']) & (economy_df[col]<=category['end']),col] = key
    for key, category in economy_categories.items():
        economy_df.loc[economy_df[col]==key,col] = category['name']
def get_economy_stats(category):

    wins_by_side_t1 = pd.DataFrame([[0,0,0],[0,0,0]],index=['ct','t'],columns=['sum','count','mean'])
    wins_by_side_t2 = pd.DataFrame([[0,0,0],[0,0,0]],index=['ct','t'],columns=['sum','count','mean'])

    for _round in range(2,16):
        gb_1 = economy_df[economy_df[str(_round)+'_t1']==category].rename(columns={'t1_start':'side'}).groupby('side')[str(_round)+'_winner']
        gb_1 = gb_1.agg(['sum','count','mean'])

        gb_3 = economy_df[economy_df[str(_round+15)+'_t1']==category].rename(columns={'t2_start':'side'}).groupby('side')[str(_round+15)+'_winner']
        gb_3 = gb_3.agg(['sum','count','mean'])

        gb_1 = gb_1.reindex(['ct','t'], fill_value=0)
        gb_3 = gb_3.reindex(['ct','t'], fill_value=0)

        wins_by_side_t1 = wins_by_side_t1 + gb_1 + gb_3

    wins_by_side_t1['sum'] = 2*wins_by_side_t1['count']-wins_by_side_t1['sum']

    for _round in range(2,16):
        gb_2 = economy_df[economy_df[str(_round)+'_t2']==category].rename(columns={'t2_start':'side'}).groupby('side')[str(_round)+'_winner']
        gb_2 = gb_2.agg(['sum','count','mean'])

        gb_4 = economy_df[economy_df[str(_round+15)+'_t2']==category].rename(columns={'t1_start':'side'}).groupby('side')[str(_round+15)+'_winner']
        gb_4 = gb_4.agg(['sum','count','mean'])

        gb_2 = gb_2.reindex(['ct','t'], fill_value=0)
        gb_4 = gb_4.reindex(['ct','t'], fill_value=0)

        wins_by_side_t2 = wins_by_side_t2 + gb_2 + gb_4

    wins_by_side_t2['sum'] = wins_by_side_t2['sum']-wins_by_side_t2['count']

    wins_by_side = wins_by_side_t1 + wins_by_side_t2

    wins_by_side['mean'] = wins_by_side['sum']/wins_by_side['count']//0.001/10
    wins_by_side['num_per_game'] = wins_by_side['count']/economy_df.shape[0]//0.1/10
    wins_by_side = wins_by_side[['mean','num_per_game']]
    
    return wins_by_side
economy_stats = {}
mean_victories_df = pd.DataFrame(index=['ct','t'])
num_per_game_df = pd.DataFrame(index=['ct','t'])

for category in economy_categories.values():
    cat = category['name']
    economy_stats[cat] = get_economy_stats(cat)
    mean_victories_df[cat] = economy_stats[cat]['mean']
    num_per_game_df[cat] = economy_stats[cat]['num_per_game']

print('\nVictory probability (%):')
display(mean_victories_df)
print('\nNumber per game:')
display(num_per_game_df)